In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import json
import os

In [2]:
!pip install steamreviews
!pip install requests

In [4]:
import requests
import steamreviews

In [5]:
response = requests.get("https://api.steampowered.com/ISteamApps/GetAppList/v2/")
appList = pd.DataFrame(response.json()['applist']['apps'])
appLists = appList['appid'].values

In [6]:
request_params = dict()
# Reference: https://partner.steamgames.com/doc/store/localization#supported_languages
request_params['language'] = 'english'
steamreviews.download_reviews_for_app_id_batch(appLists, chosen_request_params=request_params)

Loading idprocessed_on_20221031.txt
Creating idprocessed_on_20221031.txt
[appID = 1897482] num_reviews = 0 (expected: -1)
[appID = 2112761] num_reviews = 0 (expected: -1)
[appID = 1829051] num_reviews = 0 (expected: -1)
[appID = 1983382] num_reviews = 0 (expected: -1)
[appID = 216938] num_reviews = 0 (expected: -1)
[appID = 660010] num_reviews = 0 (expected: -1)
[appID = 660130] num_reviews = 0 (expected: -1)
[appID = 1118314] num_reviews = 0 (expected: -1)
[appID = 1275822] num_reviews = 0 (expected: -1)
[appID = 1343832] num_reviews = 0 (expected: -1)
[appID = 1828741] num_reviews = 0 (expected: -1)
[appID = 662172] num_reviews = 0 (expected: -1)
[appID = 1360782] num_reviews = 0 (expected: -1)
[appID = 1820332] num_reviews = 0 (expected: -1)
[appID = 1927051] num_reviews = 0 (expected: -1)
[appID = 1496152] num_reviews = 0 (expected: -1)
[appID = 1808781] num_reviews = 0 (expected: -1)
[appID = 1977312] num_reviews = 0 (expected: -1)
[appID = 1700632] num_reviews = 0 (expected: -1)


KeyboardInterrupt: 

In [ ]:
json_response = json.load(open("/kaggle/working/data/review_329070.json"))

In [ ]:
reviews = json_response['reviews']

In [ ]:
reviews

In [ ]:
# import requests
response = requests.get("https://api.steampowered.com/ISteamApps/GetAppList/v2/")
appList = pd.DataFrame(response.json()['applist']['apps'])

In [ ]:
example = requests.get("https://store.steampowered.com/appreviews/329070?json=1")

In [ ]:
appLists = appList['appid'].values

In [ ]:
appLists[:10]